In [1021]:
([1177.08 -1257.08; -1257.08 1342.52]) - (([0.5 -1 99; -0.5 -1 99] * [1/3; 1/3; 1/3] * [0.210517; -0.219267]' + [0.5 -1 99; -0.5 -1 99] * [1; 0; 0] * [48.5192; -51.817]' + [50 0 1; -50 0 1] * [1/3; 1/3; 1/3] * [0.210517; -0.219267]' + [50 0 1; -50 0 1] * [1; 0; 0] * [48.5192; -51.817]') ./ 2)

2×2 Array{Float64,2}:
 -53.2752  56.7627
 -33.6719  35.9132

In [1156]:
sum([1 2 3; 4 5 6], dims=1)

1×3 Array{Int64,2}:
 5  7  9

In [1248]:
using StatsBase
using LinearAlgebra
using Distributions
include("FHMMs.jl")

K=3
M=2
D=2
T=2

pi = [0.1 0.3; 0.8 0.2; 0.1 0.5]

C = [ 0.2 0.1; 0.1 0.2 ]

P = zeros(K,K,M)
P[:,:,1] = [ 0.25 0.35 0.3; 0.15 0.2 0.65; 0.15 0.65 0.2 ]
P[:,:,2] = [ 0.99 0.005 0.005; 0.7 0.2 0.1; 0.3 0.3 0.4 ]

W = zeros(D,K,M)
W[:,:,1] = [0.5 -1 99; -0.5 -1 99]
W[:,:,2] = [50 0 1; -50 0 1]

model = Main.FHMMs.FHMM(D,K,M,    W, pi, P, C)

#Y_n, states_n = generate(model, T)
#display(Y)
Y = [ 0.210517 48.5192; -0.219267 -51.817]

h, γ, ξ = Main.FHMMs.fit_sv2_start!(model, Y)

"C⁻¹"

2×2 Array{Float64,2}:
  6.66667  -3.33333
 -3.33333   6.66667

2-element Array{Float64,1}:
 -69.18948300000001
  10.580733

2-element Array{Float64,1}:
 -69.18948300000001
  10.580733

2-element Array{Float64,1}:
  28.169199999999996
 -51.667

...

"h new after subtracting max"

3×2×2 Array{Float64,3}:
[:, :, 1] =
   -593.38       0.0
      0.0     -321.688
 -52202.9   -40821.0

[:, :, 2] =
 -64885.1         0.0
      0.0    -14918.1
   -198.696  -14999.8

"h new after exp"

3×2×2 Array{Float64,3}:
[:, :, 1] =
 1.98716e-258  1.0
 1.0           1.96125e-140
 0.0           0.0

[:, :, 2] =
 0.0          1.0
 1.0          0.0
 5.09913e-87  0.0

([1.987161673322776e-258 1.0; 1.0 1.9612454121865234e-140; 0.0 0.0]

[0.0 1.0; 1.0 0.0; 5.099129083204362e-87 0.0], [0.2 0.5 0.3; 0.8 0.1 0.1]

[0.5 0.4 0.1; 0.2 0.2 0.6], [0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 0.0])

In [1225]:
display(model.P)

3×3×2 Array{Float64,3}:
[:, :, 1] =
 0.333333  0.333333  0.333333
 0.0       1.0       0.0
 0.333333  0.333333  0.333333

[:, :, 2] =
 5.12897e-23   0.500097  0.499903
 2.15111e-124  0.0       0.0
 2.28324e-210  0.0       0.0

In [1195]:
for i=1:1000
    Y_n, states_n = generate(model, T)
    h, γ, ξ = Main.FHMMs.fit_sv2!(model, Y_n, h, γ, ξ)
end

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [1194]:


function generate(model::Main.FHMMs.FHMM, timesteps::Int)
    X = zeros(size(model.W)[1], timesteps)
    # choose initial states based on π
    states = zeros(Int64, model.M, timesteps)
    for m=1:model.M
        idx = sample(weights(model.π[:,m]))
        states[m,1] = Int64(idx)
    end
    for t=1:timesteps    
        mu = zeros(model.D)
        for m=1:model.M
            mu += model.W[:,states[m,t],m]
        end
        n = MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C))))
        X[:,t] = rand(n)
        if t < timesteps
            for m=1:model.M
                nsp = model.P[states[m,t],:,m]
                smpl = sample(weights(nsp))
                states[m,t+1] = smpl
            end
        end
    end
    return X,states
end

generate (generic function with 73 methods)

In [799]:
function sample_states(model::Main.FHMMs.FHMM, timesteps::Int, iter=1000)
    # we need to sample theta, KxMxT
    # θ[1,1,2] represents the probabilities of chain (1) being in state (1) at time 2, 
    # θ[3,1,2] represents the probabilities of chain (1) being in state (3) at time 2, etc
    # this is unconditional wrt the observation
    theta_sampled = zeros(model.K,model.M, timesteps)
    for i=1:iter
        theta_iter = zeros(model.K,model.M,timesteps)
        for m=1:model.M
            s = sample(weights(model.π[:,m]))
            theta_iter[s, m,1] += 1
        end
        for t=2:timesteps
            for m=1:model_actual.M
                lastState = findfirst((t) -> t == 1, theta_iter[:,m,t-1])
                s = sample(weights(model.P[lastState,:,m]))
                theta_iter[s,m,t] += 1
            end
        end
        theta_sampled .+= theta_iter
    end
    theta_sampled ./= iter
    return theta_sampled
end


sample_states (generic function with 136 methods)

In [800]:
function get_state_combinations(M::Int, K::Int)
    state_combinations = []
    m = 1
    stack = []
    while(true)
        if(m == M)
            if(size(stack)[1] != M - 1)
                break
            end
            for s=1:K
                push!(stack, s)
                push!(state_combinations, copy(stack))
                pop!(stack)
            end
            stack[end] +=1
            while(stack[end] > K)
                pop!(stack)
                if(size(stack)[1] == 0)
                    break
                end
                stack[end] +=1
                m -= 1
            end
        else
            push!(stack, 1)
            m += 1
        end
    end
    return state_combinations
end

get_state_combinations (generic function with 1 method)

In [801]:
# we want to calculate 
# P(Mt=St|Yt)
# = P(Yt|Mt=St).P(Mt=St|Mt-1=St-1) / P(Yt)
# This decoding is specific to the completely factorized VI model which uses a simplified P(Mt|Yt) that assumes 
# no dependence between timesteps. Viterbi decoding is therefore not required, since we don't need to find the max sequence,
# we just take the max P at each timestep.
# even though each chain is independent but the observation is conditional on *all* chains 
# this is because Yt is a sum of M1,M2,M3...
# i.e. P(Yt|M1t=S1t,M2t=.., M3t=..) != P(Yt|M1t=S1t)
# but we have θ, which is an estimation of P(M1t=S1t), P(M1t=S2t)...P(Mmt=Sst)
# so we need to find all configurations of M/S at this t
# cross product of (M1xS)x(M2xS)x..x(MkxS)
# if we are using the variational inference model, θ is KxMxT
# θ[1,1,2] represents p(chain (1) in state (1) at time 2) 
# θ[3,1,2] represents p(chain (1) in state (3) at time 2), etc
# this is unconditional wrt the observation
# i.e. P(Mt=St|M(t-1)=S(t-1))
# therefore P(Mt=St) is the product over dim(1) at T
function decode(model::Main.FHMMs.FHMM, X::Array{Float64,2}, θ::Array{Float64,3})
    
    D=size(X)[1]
    timesteps=size(X)[2]
    T = timesteps
    
    states = zeros(model.M,timesteps)
    for t=1:timesteps
        pmax = -1
        mmax = []
        stack = []
        m = 1
        M = model.M
        K = model.K
        while(true)
            if(m == M)
                if(size(stack)[1] != M - 1)
                    break
                end
                for s=1:K
                    push!(stack, s)
                    mu = zeros(D)
                    for i=1:M
                        mu += model.W[:, stack[i], i]
                    end
                    
                    p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                    p_mt_in_s = reduce(*, θ[stack,:,t])
                    unnormalized = p_yt_given_mt_in_s * p_mt_in_s
                    if unnormalized > pmax
                        pmax = unnormalized
                        mmax = copy(stack)
                    end
                    pop!(stack)
                end
                stack[end] +=1
                while(stack[end] > K)
                    pop!(stack)
                    if(size(stack)[1] == 0)
                        break
                    end
                    stack[end] +=1
                    m -= 1
                end
            else
                push!(stack, 1)
                m += 1
            end
        end
        states[:,t] = mmax
    end
    return states
end

decode (generic function with 68 methods)

In [802]:
function decode_full(model::Main.FHMMs.FHMM, X::Array{Float64,2})
    D=size(X)[1]
    timesteps=size(X)[2]
    state_combinations = get_state_combinations(model.M, model.K)
    state_probabilities = zeros(size(state_combinations)[1],timesteps)
    
    M = model.M
    K = model.K
    logP = log.(model.P)
    for t=1:timesteps
        for s_index=1:size(state_combinations)[1]
            state = state_combinations[s_index]
            mu = zeros(D)
            for i=1:model.M
                mu += model.W[:, state[i], i]
            end
            p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                   
            if(t == 1)
               p_mt_in_s = reduce(*, model.π[[CartesianIndex(state[i], i)  for i=1:model.M]])
            else
                # calculate the probability that chain M is in state K, P(Mt)
                # this is P(Mt|Mt-1, Yt), so P(Mt-1|Yt) * P(Mt|Mt-1)
                # P(Mt-1|Yt) is sum over K
                p_mt_in_s = 0
                for sc_last_index=1:size(state_combinations)[1]
                    sc_last = state_combinations[sc_last_index]
                    for m_last=1:M
                        for m_this=1:M
                            p_mt_in_s += exp(
                                state_probabilities[sc_last_index,t-1] + 
                                logP[sc_last[m_this], state[m_this], m_this])
                        end
                    end
                end
            end
            state_probabilities[s_index,t] = log(p_yt_given_mt_in_s) +  log(p_mt_in_s)
         end
    end
    max_probs = zeros(M, timesteps)
    i = 1
    for p in argmax(state_probabilities, dims=1)
        max_probs[:, p[2]] = state_combinations[p[1]]
        i += 1
    end
    return max_probs
end


decode_full (generic function with 68 methods)

In [803]:
# setup our known model
# 
D=6
K=3
M=2
#T=5
W=cat( [ 1.0 49.0; -1.0 -49.0 ],[ 1.0 49.0; -1.0 -49.0 ], [ 1.0 49.0; -1.0 -49.0 ], dims=3)
π = [ 0.35 0.25 0.95; 0.65 0.75 0.05; ]
P = cat([ 0.3 0.7 ; 0.2 0.8 ; ],
    [ 0.5 0.5 ; 0.4 0.6 ;  ],
    [ 0.75 0.25 ; 0.89 0.11 ; ], dims=3)
W = rand(D,K,M)
π = rand(K,M)
π ./= sum(π,dims=(1))
P = rand(K, K, M)
P ./= sum(P, dims=2)
#C = rand(D,D)
C = Array{Float64}(Diagonal([0.1 for x in 1:D]))
model_actual = Main.FHMMs.FHMM(D,K,M,
    W, π, P, C)

X, actual = generate(model_actual, 4)



([1.2963597090764072 1.1606759062138554 1.6865151425697347 0.6716119434999178; 0.9839792325446926 -0.07496810994898206 0.871210678659229 1.1124955340674547; … ; 0.4457384533413633 1.9637295031609254 2.178485919888562 1.712512544543915; 1.1382162515375085 1.1775117933222603 1.6346955566965595 1.2652243512414156], [3 2 2 1; 1 2 2 2])

In [805]:
#Y = rand(5,4)

Main.FHMMs.fit_sv2!(model_actual, X)

-1257.9116607124058

-1257.9116607124058

-1257.9116607124058

-1257.9116607124058

-1257.9116607124058

...

-1257.9116607124058

-1257.9116607124058

-1257.9116607124058

-1257.9116607124058

2×3×4 Array{Float64,3}:
[:, :, 1] =
 0.894969  3.15761e-5  0.105
 1.0       4.10316e-9  1.46408e-7

[:, :, 2] =
 0.278484  3.5913e-6    0.721513
 1.0       2.88559e-10  4.58741e-7

[:, :, 3] =
 0.183086  7.06429e-6   0.816907
 1.0       3.73287e-10  4.45538e-7

[:, :, 4] =
 0.691224  1.53917e-10  0.308776
 0.999999  5.95525e-11  1.39523e-6

In [790]:
actual

2×4 Array{Int64,2}:
 3  2  1  1
 3  1  1  1

In [617]:
X, actual = generate(model_actual, 4)
model_complete = Main.FHMMs.FHMM(D,K,M)
result = Main.FHMMs.fit!(model_complete,X,maxiter=5000,verbose=true)    
decode_full(model_complete, X)

#1: score -153.97293769643957
7.147611993016899e-5: converged at #4.
#1: bound 0.0 improvement: -Inf
#2: score -84.73110081338115
0.0: converged at #3.
#2: bound -153.97293769643957 improvement: 0.44970134309946047
#3: score 787.9900923859408
0.0: converged at #1.
#3: bound -84.73110081338115 improvement: 10.299892068220334
#4: score 787.9900923859408
0.0: converged at #1.
#4: bound 787.9900923859408 improvement: 0.0
#4 converged


LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [573]:
actual

2×4 Array{Int64,2}:
 1  1  1  1
 2  3  2  3

In [ ]:
timesteps=10
num_observations = 100

theta_sampled = sample_states(model_actual, timesteps, 1000)
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode(model_actual,X, theta_sampled)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode_full(model_actual,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 50
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100)    
    
    estimated_states = decode_full(model_learn,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
timesteps=10
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 100
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100,fzero=true)    
    for i in 1:D
        model_learn.C[i, i] += 0.0001
    end
    #println(cholesky(Hermitian(Symmetric(model_learn.C))))
    estimated_states = decode(model_learn,X, result.θ)
    j += sum(actual_states .== estimated_states)
    println(j / (i * timesteps * model_actual.M))
end
j / (num_observations * timesteps * model_actual.M)